In [93]:
from collections import Counter
import operator
import re
import psycopg2 as pg2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import ttest_ind, ttest_ind_from_stats, probplot
import pylab
import statsmodels.stats.api as sms

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk.corpus import stopwords
import string

In [7]:
conn = pg2.connect(dbname='dsjob', host='localhost',
                    password='docker', user='postgres')
conn.autocommit = True
cur = conn.cursor()

cur.execute("SELECT * FROM indeed;")
l=[]
for item in cur.fetchall():
    l.append(item)

df=pd.DataFrame(l, columns=["job_jk", "job_title",  "job_location",  "company_name", "salary", "description"])

In [8]:
df.head()

,job_jk,job_title,job_location,company_name,salary,description
0,1bce25f0b7e3723d,"Senior Data Scientist – Image Analytics, Novar...","Cambridge, MA",Novartis,No salary given,\n\nWe believe the answers are found when curi...
1,1486880d275ce7c1,Senior Scientist Deep Data Analytics,"Cambridge, MA 02142",Sanofi,No salary given,Sanofi is a global life sciences company commi...
2,7d6ac302ac846400,Senior Manager of Data Science – Cyber Security,"Boston, MA 02298",No company info,No salary given,\nPrimary Responsibilities:\n Product ownershi...
3,3100f3d85650c469,Enterprise Architect Data,"Boston, MA",MassMutual,No salary given,\nA career with us means you will work alongsi...
4,3de16480e89bcaa2,Post Doctoral Fellow,"Cambridge, MA",Eisai,No salary given,"\n\nAt G2D2, we focus not only on how such dat..."


In [9]:
vectorizer = TfidfVectorizer(stop_words = "english", max_features = None)
X = vectorizer.fit_transform(df['description'])
nmf = NMF(n_components=10, random_state =0)
W= nmf.fit_transform(X)
H = nmf.components_

In [10]:
W.shape, H.shape

((14473, 10), (10, 48220))

In [21]:
vocab = vectorizer.get_feature_names()
words_per_topic = []
for i in range(H.shape[0]):
    l=[]
    for e in H[i].argsort()[::-1][0:10]:
        l.append(vocab[e])
    print(l)
    words_per_topic.append(l)
#words_per_topic    

['learning', 'machine', 'data', 'models', 'algorithms', 'ai', 'ml', 'experience', 'science', 'deep']
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'digital', 'teams', 'experience']
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
['software', 'experience', 'cloud', 'systems', 'aws', 'development', 'design', 'technical', 'data', 'technologies']
['accenture', 'cloud', 'data', 'client', 'azure', 'visa', 'clients', 'delivery', 'gcp', 'employment']
['deloitte', 'clients', 'people', 'cognitive', 'business', 'client', 'tips', 'learn', 'help', 'offer']
['iqvia', 'healthcare', 'takes', 'clinical', 'reasonable', 'accommodation', 'patients', 'clients', 'health', 'client']
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']
['status', 'employment', 'microsoft', 'protected', 'qualified', 'applicants', 'disability', 'gender', 'information', 'national']
[

In [25]:
def max_feature_topic_search(m_feature_n, topic_n):
    for m in m_feature_n:
        for n in topic_n:
            vectorizer = TfidfVectorizer(stop_words = "english", max_features = m)
            X = vectorizer.fit_transform(df['description'])
            nmf = NMF(n_components=n, random_state =0)
            W= nmf.fit_transform(X)
            H = nmf.components_
            vocab = vectorizer.get_feature_names()
            words_per_topic = []
            print ("max features = {}, n of topics ={}".format(m, n))
            for i in range(H.shape[0]):
                l=[]
                for e in H[i].argsort()[::-1][0:10]:
                    l.append(vocab[e])
                print("topic {}".format(i))
                print(l)
                words_per_topic.append(l)
    

In [26]:
m_feature_n =[20000, 10000, 5000, 2000, 1000]
topic_n = [40, 20, 10, 8, 6, 4]

max_feature_topic_search(m_feature_n, topic_n)

max features = 20000, n of topics =40
topic 0
['reporting', 'bi', 'business', 'data', 'reports', 'analysis', 'ability', 'dashboards', 'tableau', 'sql']
topic 1
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'application', 'workday_recruiting', 'brave', 'arrange', 'ultimately']
topic 2
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'school', 'date']
topic 3
['software', 'systems', 'development', 'engineering', 'engineers', 'test', 'code', 'engineer', 'technical', 'infrastructure']
topic 4
['accenture', 'visa', 'client', 'clients', 'employment', 'authorization', 'status', 'delivery', 'technology', 'business']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'tips', 'learn', 'help', 'confident', 'offer', 'client']
topic 6
['learning', 'machine', 'ai', 'ml', 'algorithms', 'deep', 'models', 'science', 'nlp', 'tensorflow']
topic 7
['intelligence', 'security', 'll', 'clearance', 'warfighters', 'abroad', 'information',

max features = 20000, n of topics =8
topic 0
['data', 'business', 'analytics', 'experience', 'analysis', 'models', 'statistical', 'modeling', 'science', 'tools']
topic 1
['product', 'marketing', 'business', 'team', 'customer', 'sales', 'work', 'strategy', 'teams', 'management']
topic 2
['research', 'university', 'teaching', 'faculty', 'health', 'position', 'department', 'students', 'information', 'program']
topic 3
['experience', 'software', 'cloud', 'learning', 'aws', 'amazon', 'systems', 'machine', 'engineering', 'development']
topic 4
['accenture', 'cloud', 'data', 'azure', 'client', 'visa', 'employment', 'delivery', 'clients', 'status']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'client', 'learn', 'business', 'tips', 'help', 'offer']
topic 6
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'clients', 'health', 'client']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'data', 'booz', 'allen', 'training', 'access', 'change']


max features = 10000, n of topics =10
topic 0
['business', 'management', 'ability', 'project', 'analytics', 'skills', 'experience', 'financial', 'analysis', 'work']
topic 1
['product', 'marketing', 'team', 'customer', 'sales', 'products', 'customers', 'business', 'work', 'teams']
topic 2
['research', 'university', 'teaching', 'faculty', 'students', 'department', 'position', 'health', 'science', 'edu']
topic 3
['software', 'experience', 'cloud', 'systems', 'development', 'aws', 'design', 'technical', 'services', 'security']
topic 4
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'clinical', 'patients', 'sales', 'health', 'clients']
topic 5
['deloitte', 'people', 'clients', 'cognitive', 'learn', 'tips', 'client', 'help', 'offer', 'confident']
topic 6
['learning', 'machine', 'models', 'algorithms', 'experience', 'ml', 'ai', 'techniques', 'statistical', 'data']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'information', 'dat

max features = 5000, n of topics =20
topic 0
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'reporting', 'risk', 'amp']
topic 1
['team', 'work', 'll', 'company', 'people', 'help', 'world', 'best', 'make', 'insurance']
topic 2
['university', 'research', 'teaching', 'faculty', 'students', 'department', 'edu', 'undergraduate', 'graduate', 'position']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'technical', 'computer', 'web']
topic 4
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'clients', 'application', 'workday_recruiting', 'brave', 'client']
topic 5
['deloitte', 'clients', 'people', 'cognitive', 'tips', 'client', 'learn', 'help', 'confident', 'offer']
topic 6
['learning', 'machine', 'models', 'algorithms', 'techniques', 'statistical', 'experience', 'data', 'ml', 'ai']
topic 7
['ll', 'intelligence', 'clearance', 'security', 'booz', 'allen', 'access', 'training', 'change', 'sci']
to

max features = 2000, n of topics =20
topic 0
['data', 'experience', 'analytics', 'sql', 'science', 'sets', 'tools', 'big', 'sources', 'analysis']
topic 1
['university', 'teaching', 'faculty', 'students', 'research', 'department', 'edu', 'undergraduate', 'graduate', 'college']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'best', 'world', 'make', 'build']
topic 3
['software', 'experience', 'development', 'systems', 'design', 'engineering', 'test', 'computer', 'web', 'technical']
topic 4
['business', 'analytics', 'management', 'ability', 'analytical', 'skills', 'analysis', 'risk', 'reporting', 'amp']
topic 5
['iqvia', 'healthcare', 'takes', 'reasonable', 'accommodation', 'patients', 'application', 'clients', 'workday_recruiting', 'brave']
topic 6
['status', 'employment', 'protected', 'gender', 'qualified', 'microsoft', 'disability', 'applicants', 'origin', 'race']
topic 7
['deloitte', 'clients', 'people', 'cognitive', 'client', 'tips', 'learn', 'help', 'confident', 'offer']

max features = 1000, n of topics =20
topic 0
['data', 'big', 'science', 'pipelines', 'sql', 'analytics', 'etl', 'tools', 'sets', 'sources']
topic 1
['university', 'teaching', 'faculty', 'students', 'department', 'edu', 'graduate', 'undergraduate', 'college', 'position']
topic 2
['team', 'll', 'work', 'people', 'company', 'help', 'world', 'best', 'make', 'love']
topic 3
['software', 'experience', 'development', 'design', 'systems', 'engineering', 'code', 'test', 'web', 'testing']
topic 4
['learning', 'machine', 'ai', 'models', 'ml', 'algorithms', 'deep', 'science', 'data', 'techniques']
topic 5
['iqvia', 'healthcare', 'takes', 'accommodation', 'reasonable', 'application', 'patients', 'clients', 'impact', 'disabled']
topic 6
['status', 'employment', 'gender', 'protected', 'disability', 'qualified', 'applicants', 'origin', 'race', 'religion']
topic 7
['deloitte', 'clients', 'people', 'client', 'learn', 'help', 'offer', 'professionals', 'culture', 'communities']
topic 8
['statistical', 'an

In [107]:
token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())



In [109]:
len(token)

9170386

In [108]:
lemmatizer = WordNetLemmatizer()

description_lemma = [lemmatizer.lemmatize(e) for e in token]

In [110]:
len(description_lemma)
description_lemma[0:100]

['we',
 'believe',
 'the',
 'answer',
 'are',
 'found',
 'when',
 'curious',
 ',',
 'courageous',
 'and',
 'collaborative',
 'people',
 'like',
 'you',
 'are',
 'brought',
 'together',
 'in',
 'an',
 'inspiring',
 'environment',
 '.',
 'where',
 'you',
 '’',
 're',
 'given',
 'opportunity',
 'to',
 'explore',
 'the',
 'power',
 'of',
 'digital',
 'and',
 'data',
 '.',
 'where',
 'you',
 '’',
 're',
 'empowered',
 'to',
 'risk',
 'failure',
 'by',
 'taking',
 'smart',
 'risk',
 ',',
 'and',
 'where',
 'you',
 '’',
 're',
 'surrounded',
 'by',
 'people',
 'who',
 'share',
 'your',
 'determination',
 'to',
 'tackle',
 'the',
 'world',
 '’',
 's',
 'toughest',
 'medical',
 'challenge',
 '.',
 'we',
 'are',
 'novartis',
 '.',
 'join',
 'u',
 'and',
 'help',
 'u',
 're-imagine',
 'medicine',
 '.',
 'two',
 'company',
 'and',
 'one',
 'incredible',
 'alliance',
 '.',
 'novartis',
 'and',
 'microsoft',
 'have',
 'formed',
 'alliance',
 'to',
 'leverage']

In [ ]:
def tokenize_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    token = nltk.word_tokenize(" ".join(df['description'].map(str)).lower())
    return [lemmatizer.lemmatize(e) for e in token]
    
    
tokenize_lemmatize(" ".join(df['description'].map(str)).lower())

In [111]:
len(set(description_lemma))

74435

In [112]:
vocab_lem = set(description_lemma)

In [127]:
bigrams = ngrams(description_lemma,2)

fourgrams = ngrams(description_lemma,4)
fivegrams = ngrams(description_lemma,5)

In [114]:
[e for e in bigrams][0:10]

[('we', 'believe'),
 ('believe', 'the'),
 ('the', 'answer'),
 ('answer', 'are'),
 ('are', 'found'),
 ('found', 'when'),
 ('when', 'curious'),
 ('curious', ','),
 (',', 'courageous'),
 ('courageous', 'and')]

In [116]:
bigrams_sort = sorted(Counter(bigrams).items(), key = operator.itemgetter(1))

In [117]:
print(bigrams_sort[-1000:])

[(('or', 'local'), 943), (('will', 'help'), 945), (('unstructured', 'data'), 945), (('all', 'of'), 946), (('to', 'find'), 946), (('and', 'create'), 948), (('source', ','), 948), (('and', 'use'), 948), (('a', 'member'), 949), (('in', 'all'), 950), (('background', 'in'), 950), ((',', 'it'), 951), (('be', 'the'), 951), (('practice', ','), 951), (('and', 'implementing'), 952), (('a', 'disability'), 953), (('is', 'not'), 954), (('and', 'operation'), 954), (('with', 'internal'), 955), (('.', 'required'), 957), (('(', 's'), 957), (('business', 'process'), 957), (('business', 'development'), 957), (('business', 'requirement'), 958), (('’', 'experience'), 958), (('strategy', '.'), 959), (('life', '.'), 960), (('believe', 'that'), 960), ((':', 'you'), 960), (('them', 'to'), 961), (('technique', ','), 962), (('.', 'and'), 962), (('scientist', 'and'), 964), (('and', 'building'), 964), (('designing', 'and'), 965), (('with', 'all'), 965), (('the', 'work'), 966), ((',', 'medical'), 968), (('language'

In [68]:
len(bigrams_sort)

880314

In [151]:
my_stop_words = stopwords.words('english')
len(my_stop_words)


179

In [152]:
my_stop_words.extend(list(string.punctuation))
my_stop_words.extend(['--'])
len(my_stop_words)

212

In [120]:

bigrams_sort_sw_rm = [(u,v) for (u,v),k in bigrams_sort if (u not in my_stop_words) and (v not in my_stop_words)]

In [121]:
len(bigrams_sort_sw_rm)

454067

In [122]:
bigrams_sort_sw_rm[-1000:]

[('job', 'seeker'),
 ('solution', 'across'),
 ('7', 'year'),
 ('experience', 'creating'),
 ('enjoy', 'working'),
 ('technology', 'including'),
 ('service', 'men'),
 ('collaboration', 'skill'),
 ('control', 'system'),
 ('request', 'via'),
 ('developing', 'software'),
 ('high', 'performing'),
 ('benefits/perks', 'listed'),
 ('financial', 'strength'),
 ('business', 'owner'),
 ('data', 'architect'),
 ('deep', 'technical'),
 ('professional', 'skill'),
 ('solution', 'design'),
 ('assistant', 'professor'),
 ('problem', 'using'),
 ('leadership', 'role'),
 ('team', "'s"),
 ('conduct', 'research'),
 ('management', 'information'),
 ('applied', 'statistic'),
 ('technical', 'assistance'),
 ('using', 'machine'),
 ('ensure', 'data'),
 ('tight', 'deadline'),
 ('statistical', 'programming'),
 ('request', 'form'),
 ('data', 'access'),
 ('advanced', 'knowledge'),
 ('financial', 'institution'),
 ('customer', 'relationship'),
 ('in-depth', 'knowledge'),
 ('stakeholder', 'across'),
 ('data', 'insight'),
 ('

In [153]:
trigrams = ngrams(description_lemma,3)
trigrams_sort = sorted(Counter(trigrams).items(), key = operator.itemgetter(1), reverse = True)

trigrams_sort_sw_rm = [((u,v, w),k) for (u,v, w),k in trigrams_sort if (u not in my_stop_words) and (v not in my_stop_words) and (w not in my_stop_words)]


In [154]:
len(trigrams_sort_sw_rm)

469593

In [155]:
trigrams_sort_sw_rm[:400]

[(('equal', 'opportunity', 'employer'), 4046),
 (('employment', 'without', 'regard'), 2319),
 (('bachelor', "'s", 'degree'), 1788),
 (('protected', 'veteran', 'status'), 1646),
 (('equal', 'employment', 'opportunity'), 1573),
 (('data', 'science', 'team'), 1550),
 (('written', 'communication', 'skill'), 1400),
 (('large', 'data', 'set'), 1014),
 (('natural', 'language', 'processing'), 935),
 (('year', '’', 'experience'), 921),
 (('machine', 'learning', 'model'), 886),
 (('verbal', 'communication', 'skill'), 879),
 (('subject', 'matter', 'expert'), 861),
 (('master', "'s", 'degree'), 834),
 (('machine', 'learning', 'algorithm'), 753),
 (('machine', 'learning', 'technique'), 743),
 (('proven', 'track', 'record'), 677),
 (('opportunity/affirmative', 'action', 'employer'), 594),
 (('affirmative', 'action', 'employer'), 590),
 (('equal', 'opportunity/affirmative', 'action'), 571),
 (('excellent', 'communication', 'skill'), 558),
 (('employer', '–', 'minority'), 556),
 (('big', 'data', 'tech

In [ ]:


trigrams = ngrams(description_lemma,3)
trigrams_sort = sorted(Counter(trigrams).items(), key = operator.itemgetter(1), reverse = True)

trigrams_sort_sw_rm = [((u,v, w),k) for (u,v, w),k in trigrams_sort if (u not in my_stop_words) and (v not in my_stop_words) and (w not in my_stop_words)]


In [163]:
def n_grams_freq(tokens, n, stopwords):
    n_grams = ngrams(tokens,n)
    n_grams_sort = sorted(Counter(n_grams).items(), key = operator.itemgetter(1), reverse = True)
    
    return ( [(k,v) for k, v in n_grams_sort if all([e not in stopwords for e in k])] )

In [164]:
n_grams_freq(description_lemma,2, my_stop_words)

[(('data', 'science'), 19085),
 (('machine', 'learning'), 12113),
 (('computer', 'science'), 7092),
 (('data', 'scientist'), 6628),
 (('sexual', 'orientation'), 6062),
 (('national', 'origin'), 5497),
 (('communication', 'skill'), 5262),
 (('gender', 'identity'), 5166),
 (('equal', 'opportunity'), 5126),
 (('big', 'data'), 4895),
 (('opportunity', 'employer'), 4554),
 (('veteran', 'status'), 4527),
 (('experience', 'working'), 4112),
 (('without', 'regard'), 3538),
 (('data', 'analysis'), 3459),
 (('related', 'field'), 3251),
 (('qualified', 'applicant'), 3197),
 (('data', 'set'), 3191),
 (('team', 'member'), 3125),
 (('best', 'practice'), 3093),
 (('bachelor', '’'), 2921),
 (('data', 'analytics'), 2693),
 (("'s", 'degree'), 2677),
 (('software', 'development'), 2602),
 (('data', 'management'), 2453),
 (('work', 'experience'), 2389),
 (('marital', 'status'), 2384),
 (('employment', 'without'), 2351),
 (('reasonable', 'accommodation'), 2323),
 (('data', 'visualization'), 2272),
 (('busi